In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport event_selector
%aimport looper_utils
%aimport make_plots

In [4]:
import awkward as ak
import uproot
import numpy as np
from yahist import Hist1D, Hist2D

## Skim Parameters

In [25]:
year = '2018'
skim_version = 'v4'  #currently available x3 or v4

## Load data

In [26]:
%%bash
ls metadata/

chunklist.json
chunklist_2016_v4.json
chunklist_2017_v4.json
chunklist_2018_v4.json
chunklist_2018_x3.json
totalWeights.json
totalWeights_2016_v4.json
totalWeights_2017_v4.json
totalWeights_2018_v4.json
totalWeights_2018_x3.json
xsection.json
xsection_2016_v4.json
xsection_2017_v4.json
xsection_2017_x3.json
xsection_2018_v4.json
xsection_2018_x3.json
xsection_v4.json


In [27]:
import json
with open('./metadata/chunklist_'+year+'_'+skim_version+'.json') as f:
    chunks = json.load(f) 
chunks.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'VH', 'signal'])

## Process data

In [28]:
def process(args):
    
    import event_selector
    import make_plots
    
    obj_list = ["electron", "muon", "tau", "photon", "others"]
   
    isData = "EGamma" in args[0] or "DoubleEG" in args[0]
    isSigLike = "signal" in args[0] or "VH" in args[0]
    events = event_selector.prepare_inputs(args, obj_list, isData=isData)
    
    tot_w = ak.sum(events.genWeight)
    mgg = events.ggMass
    #svfit = events.tautauMass_SVFit
    
    ## need to change gHidx
    gHidx = event_selector.get_gHidx(args) 
    g_Hidx = gHidx.gHidx
    mask_diphoton = event_selector.select_photon_byEvent( events.Photon, g_Hidx, mgg, isSigLike )
    
    mask_photon = event_selector.select_photon(events.Photon, g_Hidx, mgg)
    photons_selected = events.Photon[mask_photon] 
    nPho = ak.num(photons_selected)
    
    mask_tau = event_selector.select_tau(events.Tau, "all", isTight=False)
    ## clean by dR wrt photons
    mask_tau = mask_tau & looper_utils.mask_by_dR(events.Tau, photons_selected, 0.2)
    nTau = ak.num(events.Tau[mask_tau])
    
    mask_ele = event_selector.select_electron(events.Electron, isTight=False)
    mask_ele = mask_ele & looper_utils.mask_by_dR(events.Electron, photons_selected, 0.2)
    nEle = ak.num(events.Electron[mask_ele])
    
    mask_mu = event_selector.select_muon(events.Muon, isTight=False)
    mask_mu = mask_mu & looper_utils.mask_by_dR(events.Muon, photons_selected, 0.2)
    nMu = ak.num(events.Muon[mask_mu])
    
    sum_charge = (ak.sum(events.Tau[mask_tau].charge, axis=1) + ak.sum(events.Electron[mask_ele].charge, axis=1) + ak.sum(events.Muon[mask_mu].charge, axis=1) == 0)
    mask_dR_mll_0lep_2tau = looper_utils.mask_by_dR_mll_0lep_2tau(events.Tau[mask_tau], (0.2,3.5), (30,140))
    mask_dR_mll_1mu_1tau = looper_utils.mask_by_dR_mll_1lep_1tau(events.Tau[mask_tau], events.Muon[mask_mu], (0.3,3.5), (20,120))
    mask_dR_mll_1ele_1tau = looper_utils.mask_by_dR_mll_1lep_1tau(events.Tau[mask_tau], events.Electron[mask_ele], (0.3,3.5), (20,120))
    
    mask_dipho = (nPho == 2)
    mask_0lep_1tau = (nPho == 2) & (nTau == 1) & (nEle == 0) & (nMu == 0)
    mask_0lep_2tau = (nPho == 2) & (nTau == 2) & (nEle == 0) & (nMu == 0) & (sum_charge) & (mask_dR_mll_0lep_2tau)
    mask_2lep_0tau = (nPho == 2) & (nTau == 0) & (nEle + nMu == 2) & (sum_charge)
    mask_1lep_0tau = (nPho == 2) & (nTau == 0) & (nEle + nMu == 1)
    
    mask_1mu_1tau = (nPho == 2) & (nTau == 1) & (nEle == 0) & (nMu == 1) & (sum_charge) & (mask_dR_mll_1mu_1tau)
    mask_1ele_1tau = (nPho == 2) & (nTau == 1) & (nEle == 1) & (nMu == 0) & (sum_charge) & (mask_dR_mll_1ele_1tau)
    mask_1lep_1tau = (mask_1mu_1tau) | (mask_1ele_1tau)
    
    masks = {"dipho": mask_dipho,
             "0lep_1tau": mask_0lep_1tau,
             "0lep_2tau": mask_0lep_2tau,
             "1lep_1tau": mask_1lep_1tau,
             "2lep_0tau": mask_2lep_0tau,
             "1lep_0tau": mask_1lep_0tau}
    
    hists = {}
    
    for key, mask in masks.items():
        print (key)
        hists[key] = make_plots.process_event( events[mask], events.genWeight[mask] )
        #print ("process diphoton")
        hists[key].update( make_plots.process_diphoton(events.Photon[mask], g_Hidx[mask], mgg[mask], events.genWeight[mask]) )
        #hists[key] =  make_plots.process_diphoton(events.Photon[mask], g_Hidx[mask], mgg[mask], events.genWeight[mask])
        #print ("process tau")
        hists[key].update( make_plots.process_tau(events.Tau[mask_tau][mask], events.genWeight[mask]))
        #print ("process muon")
        hists[key].update( make_plots.process_muon(events.Muon[mask_mu][mask], events.genWeight[mask]))
        #print ("process electron")
        hists[key].update( make_plots.process_electron(events.Electron[mask_ele][mask], events.genWeight[mask]))
        #print ("process tau-lep")
        if key == "1lep_1tau":
            tmphists = hists[key].update( make_plots.process_1tau_1lep(events.Tau[mask_tau][mask], events.Muon[mask_mu][mask],
                                                                       events.Electron[mask_ele][mask], events.genWeight[mask]))
            #if bool(tmphists):
            #    hists[key].update(tmphists)
    
    return hists

In [15]:
%%time
hists = process(chunks["signal"][0])
#hists = process((chunks["signal"][0][0], 0, 200) )
#hists = process(chunks["ZH"][0])

/home/users/fsetti/miniconda3/envs/daskanalysisenv/lib/python3.8/site-packages/numba/core/dispatcher.py:237: UserWarning: Numba extension module 'awkward1._connect._numba' failed to load due to 'ImportError(generic_type: type "kernel_lib" is already registered!)'.
  entrypoints.init_all()


dipho
0lep_1tau
0lep_2tau
1lep_1tau
2lep_0tau
1lep_0tau
CPU times: user 9.04 s, sys: 261 ms, total: 9.3 s
Wall time: 10.5 s


In [16]:
#hists['1lep_1tau']['tautauL_SVFit']
hists['1lep_1tau']['MET']

[ 8.00 ± 2.83   25.99 ± 5.10   32.99 ± 5.74   45.00 ± 6.71   52.00 ± 7.21
 61.00 ± 7.81   73.98 ± 8.60   73.99 ± 8.60   73.00 ± 8.54   72.99 ± 8.54
 86.00 ± 9.27   87.00 ± 9.33   86.00 ± 9.27   80.00 ± 8.94   66.98 ± 8.18
 72.00 ± 8.49   46.00 ± 6.78   64.99 ± 8.06   57.98 ± 7.61   60.00 ± 7.75
 51.00 ± 7.14   41.98 ± 6.48   49.00 ± 7.00   39.99 ± 6.32   42.98 ± 6.55
 29.99 ± 5.48   40.00 ± 6.32   34.00 ± 5.83   26.00 ± 5.10   27.00 ± 5.20
 23.00 ± 4.80   27.00 ± 5.20   15.00 ± 3.87   23.00 ± 4.80   18.00 ± 4.24
 19.00 ± 4.36   23.00 ± 4.80   13.00 ± 3.61    8.00 ± 2.83   15.00 ± 3.87
  9.00 ± 3.00    9.00 ± 3.00   10.00 ± 3.16    9.00 ± 3.00    6.00 ± 2.45
  9.00 ± 3.00   10.00 ± 3.16   11.00 ± 3.32    9.00 ± 3.00    8.00 ± 2.83
  9.00 ± 3.00    5.00 ± 2.24    5.00 ± 2.24    6.00 ± 2.45    2.00 ± 1.41
  4.00 ± 2.00    4.00 ± 2.00    3.00 ± 1.73    3.00 ± 1.73    3.00 ± 1.73
  4.00 ± 2.00    3.00 ± 1.73    4.00 ± 2.00    2.00 ± 1.41    2.00 ± 1.41
  4.00 ± 2.00    3.00 ± 1.73    2.00 ± 1.41    3.00 ± 1.73    1.00 ± 1.00
  2.00 ± 1.41    2.00 ± 1.41    1.00 ± 1.00    4.00 ± 2.00    2.00 ± 1.41
  0.00 ± 0.00    2.00 ± 1.41    3.00 ± 1.73    2.00 ± 1.41    1.00 ± 1.00
  4.00 ± 2.00    0.00 ± 0.00    1.00 ± 1.00    6.00 ± 2.45    1.00 ± 1.00
  1.00 ± 1.00    2.00 ± 1.41    1.00 ± 1.00    0.00 ± 0.00    0.00 ± 0.00
  1.00 ± 1.00    2.00 ± 1.41    0.00 ± 0.00    1.00 ± 1.00    0.00 ± 0.00
  0.00 ± 0.00    3.00 ± 1.73    2.00 ± 1.41    0.00 ± 0.00    9.00 ± 3.00]

In [11]:
hists['dipho']['pho_pT1'].integral
#hists['dipho']['pho_pT1'].integral_error

16344.0

## send to dask

In [1]:
#from dask.distributed import Client
#client = Client('tcp://169.228.130.74:5608')
#client

Client Scheduler: tcp://169.228.130.74:5608 Dashboard: http://169.228.130.74:12332/status,Cluster Workers: 15 Cores: 15 Memory: 60.00 GB


In [9]:
from dask.distributed import Client
client = Client(memory_limit='4GB', n_workers=32, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:10506 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 32 Cores: 32 Memory: 128.00 GB


In [10]:
def g():
    import glob
    allpys = glob.glob("./*py")
    return allpys
g()

['./__init__.py',
 './cachepreload.py',
 './condor_utils.py',
 './event_selector.py',
 './loop.py',
 './looper_utils.py',
 './make_plots.py',
 './utils.py']

In [11]:
x = client.submit(g)

In [12]:
client.gather(x)

['./__init__.py',
 './cachepreload.py',
 './condor_utils.py',
 './event_selector.py',
 './loop.py',
 './looper_utils.py',
 './make_plots.py',
 './utils.py']

In [ ]:
%%time
from dask.distributed import as_completed
import collections, functools, operator 

futures = {}
results = {}

cat_keys = ["dipho", "0lep_1tau", "0lep_2tau", "1lep_1tau", "2lep_0tau", "1lep_0tau"]

for process_key in chunks:
    print ("start process {}".format(process_key))
    futures[process_key] = client.map(process, chunks[process_key], retries=5)
    
    results_local = []
    results[process_key] = {}
    ## do you really need this while?
    while len(results_local) < len(chunks[process_key]): 
        ac = as_completed(futures[process_key], with_results=True)
        for future, result in ac:
            results_local.append(result)
    
    ## merge histograms by the key
    for cat_key in cat_keys: 
        dicts = [results_local[i][cat_key] for i in range(len(results_local))]  
        counter = collections.Counter() 
        for d in dicts:  
            counter.update(d) 
        results[process_key][cat_key] = dict(counter) 

start process EGamma_2018A
start process EGamma_2018B
start process EGamma_2018C
start process EGamma_2018D
start process DYJets
start process ttbar
start process ZG
start process WG
start process GJets_HT40To100
start process GJets_HT100To200
start process GJets_HT200To400
start process GJets_HT400To600
start process GJets_HT600ToInf
start process QCD_pT30To40
start process QCD_pT40ToInf
start process Diphoton
start process VH
start process signal


## save to disk?

- foler1: cat (dipho, 0lep_1tau, 0lep_2tau...)
    + folder2: process (data, signal, ZG...)
        - folder3: hists (pT, eta, phi...)

In [42]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [41]:
results.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'VH', 'signal'])

In [38]:
results['signal'].keys()

dict_keys(['dipho', '0lep_1tau', '0lep_2tau', '1lep_1tau', '2lep_0tau', '1lep_0tau'])

In [39]:
results['signal']['dipho'].keys()

dict_keys(['MET', 'tautau_SVFit', 'tautauA_SVFit', 'tautauL_SVFit', 'pho_pT1', 'pho_pT2', 'pho_pTom1', 'pho_pTom2', 'pho_eta1', 'pho_eta2', 'pho_phi1', 'pho_phi2', 'pho_id1', 'pho_id2', 'tau_pT1', 'tau_eta1', 'tau_phi1', 'tau_deeptau_vs_j_1', 'tau_deeptau_vs_m_1', 'tau_deeptau_vs_e_1', 'n_tau', 'mtautau', 'dR_tautau', 'tau_pT2', 'tau_eta2', 'tau_phi2', 'tau_deeptau_vs_j_2', 'tau_deeptau_vs_m_2', 'tau_deeptau_vs_e_2', 'muon_pT1', 'muon_eta1', 'muon_phi1', 'muon_iso1', 'n_muon', 'mmumu', 'dR_mumu', 'muon_pT2', 'muon_eta2', 'muon_phi2', 'muon_iso2', 'electron_pT1', 'electron_eta1', 'electron_phi1', 'electron_iso1', 'n_electron', 'mee', 'dR_ee', 'electron_pT2', 'electron_eta2', 'electron_phi2', 'electron_iso2'])

In [24]:
%%time
from subprocess import call
import json

process_keys = results.keys()
cat_keys = results['signal'].keys()

tag = "basic_dR_mll_cut_blind"

for cat_key in cat_keys:
    for process_key in process_keys:
        dirname = './hists/' + tag + '/' + cat_key + '/' + process_key + '/'
        call('mkdir -p ' + dirname, shell=True)
        for hist_key in results[process_key][cat_key].keys():
            histname = dirname + hist_key +'_'+year+'_'+skim_version
            results[process_key][cat_key][hist_key].to_json(histname+".json")
            with open(histname, "w") as f:
                data = json.dump(results[process_key][cat_key][hist_key].to_json(histname+".json"), f)

CPU times: user 4.97 s, sys: 4.55 s, total: 9.52 s
Wall time: 14.1 s
